In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]

identity_matrix = np.zeros((2,2), dtype = 'complex128')
identity_matrix[0,0]= 1
identity_matrix[1,1] = 1

In [3]:
def norm_diff(L, M):
    LminusM = L - M
    mag_LminusM = np.sqrt(np.sum(np.abs(LminusM)**2))
    mag_L = np.sqrt(np.sum(np.abs(L)**2))
    mag_M = np.sqrt(np.sum(np.abs(M)**2))
    
    delta = mag_LminusM / (mag_L + mag_M)
    
    return delta

def matrix_maker(m0xyz): #array to matrix
    M0, Mx, My, Mz = m0xyz
    
    identity_matrix = np.zeros((2,2), dtype = 'complex128')
    identity_matrix[0,0]= 1
    identity_matrix[1,1] = 1

    pauli_x = np.zeros((2,2), dtype = 'complex128')
    pauli_x[0,1]= 1
    pauli_x[1,0] = 1

    pauli_y = np.zeros((2,2), dtype = 'complex128')
    pauli_y[0,1]= -1j
    pauli_y[1,0] = 1j

    pauli_z = np.zeros((2,2), dtype = 'complex128')
    pauli_z[0,0]= 1
    pauli_z[1,1] = -1
    
    m = np.array([Mx, My, Mz])
    
    pauli_basis = np.array([pauli_x, pauli_y, pauli_z])
    
    m_dot_pauli = np.tensordot(m, pauli_basis, axes=1)
    
    matrix = M0 * identity_matrix + m_dot_pauli
    
    return matrix

def random_complex_numbers():
    real_parts = np.random.randn(4)
    imaginary_parts = np.random.randn(4)
    complex_numbers = real_parts + 1j*imaginary_parts
    return complex_numbers

In [4]:
def matrix_mult_arrays(a0xyz, b0xyz): #arrays to array, used to be calculate_c0xyz
    A0, Ax, Ay, Az = a0xyz
    B0, Bx, By, Bz = b0xyz
    axyz = np.array([Ax, Ay, Az])
    bxyz = np.array([Bx, By, Bz])
    C0 = A0*B0 + np.tensordot(axyz, bxyz, axes=1)
    cxyz = A0*bxyz + B0*axyz +1j*(np.cross(axyz, bxyz))
    c0xyz = ([C0, cxyz[0], cxyz[1], cxyz[2]])
    return c0xyz


#functions to take p arrays and make general a and b, so that we can do multiplication noramllly
def p0xyz2_to_a0xyz(p0xyz_2): #used to be array_for_rho2
    P0, Px, Py, Pz = p0xyz_2
    pxyz = np.array([Px, Py, Pz])
    A0 = 1 - 0.5*P0
    axyz = -0.5*P0*pxyz
    a0xyz = ([A0, axyz[0], axyz[1], axyz[2]])
    return a0xyz

def p0xyz4_to_b0xyz(p0xyz_4): #used to be array_for_rho4
    P0, Px, Py, Pz = p0xyz_4
    pxyz = np.array([Px, Py, Pz])
    B0 = 0.5*P0
    bxyz = 0.5*P0*pxyz
    b0xyz = ([B0, bxyz[0], bxyz[1], bxyz[2]])
    return b0xyz #makes array to be put into matrix maker to create rho

#takes p arrays and makes matrix C (which is A*B)
def p0xyz2_p0xyz4_to_C(p0xyz_2, p0xyz_4):
    a0xyz = p0xyz2_to_a0xyz(p0xyz_2)
    b0xyz = p0xyz4_to_b0xyz(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    C = matrix_maker(c0xyz)
    return C

#two ways to make final F
def F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    a0xyz = p0xyz2_to_a0xyz(p0xyz_2)
    b0xyz = p0xyz4_to_b0xyz(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    return (f0xyz)
#note: can probably refine F_array_to_matrix function to just call F_array and then matrix maker
def F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    a0xyz = p0xyz2_to_a0xyz(p0xyz_2)
    b0xyz = p0xyz4_to_b0xyz(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    F = matrix_maker(f0xyz)
    return (F)

def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = p0xyz2_p0xyz4_to_C(p0xyz_2, p0xyz_4)
    D = C + np.trace(C)*identity_matrix
    E = matrix_maker(p0xyz_3)@D
    F = matrix_maker(p0xyz_1)@E
    return (F)

rewrite all necessary functions above
- want to get to this:

In [5]:
def Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

#def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    #F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    #F_conj = np.conjugate(F)
    #Re_F = F + F_conj
    #return Re_F

def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

def Fvvsc_arr_2_mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

In [6]:
#make matrix from array
Re_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
Fvvsc_from_array = matrix_maker(Re_f0xyz)
print('Fvvsc_array PRODUCES:')
print(Fvvsc_from_array)

Fvvsc_arr_2_mat = Fvvsc_arr_2_mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print('Fvvsc_arr_2_mat PRODUCES:')
print(Fvvsc_arr_2_mat)

F_made_with_matrices = Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print('Fvvsc_matrix PRODUCES:')
print(F_made_with_matrices)

Fvvsc_array PRODUCES:
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]
Fvvsc_arr_2_mat PRODUCES:
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]
Fvvsc_matrix PRODUCES:
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]


In [7]:
FM = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
FA = F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print(FM)
print(FA)

[[-5.+1.j -3.+1.j]
 [-1.-1.j -1.+1.j]]
[[-5.+1.j -3.+1.j]
 [-1.-1.j -1.+1.j]]


In [8]:
def Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0 = f0xyz[0]
    f0_conj = np.conjugate(f0xyz[0])
    Re_f0 = f0 + f0_conj
    print(Re_f0)
   
    fx = f0xyz[1]
    fx_conj = np.conjugate(f0xyz[1])
    Re_fx = fx + fx_conj
    print(Re_fx)
    
    fy = f0xyz[2]
    fy_conj = np.conjugate(f0xyz[2])
    Re_fy = fy + fy_conj
    print(Re_fy)
    
    
    fz = f0xyz[3]
    fz_conj = np.conjugate(f0xyz[3])
    Re_fz = fz + fz_conj
    print(Re_fz)
    print ('f0xyz', f0xyz)
    
    Re_f0xyz = [Re_f0, Re_fx, Re_fy, Re_fz]
    return Re_f0xyz, f0xyz

def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    print(F)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    #need transpose
    return Fvvsc

In [9]:
Re_f0xyz, Im_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
Re_f0xyz, Im_f0xyz

(-6+0j)
(-4+0j)
(-2+0j)
(-4+0j)
f0xyz [(-3+1j), (-2+0j), (-1-1j), (-2+0j)]


([(-6+0j), (-4+0j), (-2+0j), (-4+0j)], [(-3+1j), (-2+0j), (-1-1j), (-2+0j)])

In [10]:
Re_f0xyz
matrixA = matrix_maker(Re_f0xyz)
matrixA_im = matrix_maker(Im_f0xyz)
print(Re_f0xyz) # how is it taking all real numbers and returning a comlplex matri
matrixA, matrixA_im

[(-6+0j), (-4+0j), (-2+0j), (-4+0j)]


(array([[-10.+0.j,  -4.+2.j],
        [ -4.-2.j,  -2.+0.j]]),
 array([[-5.+1.j, -3.+1.j],
        [-1.-1.j, -1.+1.j]]))

In [11]:
matrixM = Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
matrixM

[[-5.+1.j -3.+1.j]
 [-1.-1.j -1.+1.j]]


array([[-10.+0.j,  -4.+2.j],
       [ -4.-2.j,  -2.+0.j]])

In [12]:
##################################################################################
#this is how i make complex conj in Fvvsc_array
#################################################################################
f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
f0xyz_conj = np.conjugate(f0xyz)

print('******** this is how i make complex conj in Fvvsc_array *******')
print('f0xyz=',f0xyz)
print('f0xyz_conj=', f0xyz_conj)

print('********')
f0xyz_to_matrix = matrix_maker(f0xyz)
f0xyz_conj_to_matrix = matrix_maker(f0xyz_conj)
print('F array to matrix=', f0xyz_to_matrix)
print('Complex conj F array to matrix=', f0xyz_conj_to_matrix)
print('adding those^', f0xyz_to_matrix + f0xyz_conj_to_matrix )
print('so taking f0xyz and f0xyz_conj and making matrices, they make the transpose of each other.. interesting')

print('********')
Re_f0xyz = f0xyz + f0xyz_conj
print('Re_f0xyz', Re_f0xyz)
Re_F_made_w_arrays = matrix_maker(Re_f0xyz)
print('Re_F made from array=', Re_F_made_w_arrays, "the 2j should NOT be imaginary!!! why??")



#################################################################################
#this is how i make complex conj in Fvvsc_matrix
#################################################################################
F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
F_conj = np.conjugate(F)

print('******** this is how i make complex conj in Fvvsc_matrix *******')
print('F=',F)
print('F_conj=',F_conj)
Re_F = F + F_conj
print('Re_F made directly=', Re_F)

******** this is how i make complex conj in Fvvsc_array *******
f0xyz= [(-3+1j), (-2+0j), (-1-1j), (-2+0j)]
f0xyz_conj= [-3.-1.j -2.-0.j -1.+1.j -2.-0.j]
********
F array to matrix= [[-5.+1.j -3.+1.j]
 [-1.-1.j -1.+1.j]]
Complex conj F array to matrix= [[-5.-1.j -1.+1.j]
 [-3.-1.j -1.-1.j]]
adding those^ [[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]
so taking f0xyz and f0xyz_conj and making matrices, they make the transpose of each other.. interesting
********
Re_f0xyz [-6.+0.j -4.+0.j -2.+0.j -4.+0.j]
Re_F made from array= [[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]] the 2j should NOT be imaginary!!! why??
******** this is how i make complex conj in Fvvsc_matrix *******
F= [[-5.+1.j -3.+1.j]
 [-1.-1.j -1.+1.j]]
F_conj= [[-5.-1.j -3.-1.j]
 [-1.+1.j -1.-1.j]]
Re_F made directly= [[-10.+0.j  -6.+0.j]
 [ -2.+0.j  -2.+0.j]]


# basically what is happening is that when I made $F^{\nu\nu}_{\mathrm{sc}}$ two ways, they are the same matrix, its just that when I do array to matrix maker there is an extra factor of j on the 2 on the diagonals... 

# this is because $F^{\nu\nu}_{\mathrm{sc}}$ made with the array first, makes a real array but put that into matrix maker and if it has a pauli y component the outputted matrix will have an imaginary part

testing